In [ ]:
# ! git clone https://github.com/CodeArchitecture/FinRL.git

In [ ]:
# import os
# os.chdir('/kaggle/working/FinRL')
# print(os.getcwd())

In [ ]:
! pip install -r requirements.txt

In [4]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

# import sys
# sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.9/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [5]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



In [6]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [7]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TRADE_START_DATE = '2021-10-01'
TRADE_END_DATE = '2023-03-01'

In [8]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [10]:
df.shape

(97013, 8)

In [11]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,6.505280,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,42.888943,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,33.675961,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,43.777538,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,41.156910,7325600,CAT,0


In [12]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


In [13]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [14]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.505280,493729600.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,6.505280,6.505280,20.040001,0.0
1,2010-01-04,AMGN,56.630001,57.869999,56.560001,42.888943,5277400.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,42.888943,42.888943,20.040001,0.0
2,2010-01-04,AXP,40.810001,41.099998,40.389999,33.675961,6894300.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,33.675961,33.675961,20.040001,0.0
3,2010-01-04,BA,55.720001,56.389999,54.799999,43.777538,6186700.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,43.777538,43.777538,20.040001,0.0
4,2010-01-04,CAT,57.650002,59.189999,57.509998,41.156910,7325600.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,41.156910,41.156910,20.040001,0.0
5,2010-01-04,CRM,18.652500,18.882500,18.547501,18.705000,7906000.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,18.705000,18.705000,20.040001,0.0
6,2010-01-04,CSCO,24.110001,24.840000,24.010000,17.394125,59853700.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,17.394125,17.394125,20.040001,0.0
7,2010-01-04,CVX,78.199997,79.199997,78.160004,46.851871,10173800.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,46.851871,46.851871,20.040001,0.0
8,2010-01-04,DIS,32.500000,32.750000,31.870001,27.933922,13700400.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,27.933922,27.933922,20.040001,0.0
9,2010-01-04,GS,170.050003,174.250000,169.509995,139.862900,9135000.0,0.0,0.0,6.526809,6.494997,100.0,66.666667,100.0,139.862900,139.862900,20.040001,0.0


In [15]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

In [16]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

85753
10237


In [17]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2956,2021-09-30,UNH,401.489990,403.489990,390.459991,383.180145,3779900.0,3.0,-4.349257,419.212249,386.863767,40.895372,-222.938263,41.980385,405.947460,405.438415,23.139999,24.871997
2956,2021-09-30,V,227.580002,228.789993,222.630005,220.257767,7128500.0,3.0,-1.538726,228.639288,216.529588,44.078992,-54.614561,19.569853,224.777294,231.479429,23.139999,24.871997
2956,2021-09-30,VZ,54.500000,54.509998,54.000000,49.822678,18736600.0,3.0,-0.237533,50.960816,49.550736,41.824914,-102.899147,21.682953,50.425972,50.960237,23.139999,24.871997
2956,2021-09-30,WBA,48.790001,48.930000,46.919998,43.957283,6449400.0,3.0,-0.253540,48.531811,43.597245,44.613717,-107.390242,0.941150,45.886550,44.857786,23.139999,24.871997
2956,2021-09-30,WMT,140.639999,141.729996,139.250000,136.672668,7485900.0,3.0,-1.561206,146.754148,136.189053,40.165800,-151.553868,45.466733,142.928735,141.825645,23.139999,24.871997


In [18]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,141.404266,94639600.0,4.0,-1.703487,155.382845,137.132192,46.927735,-142.190143,41.749873,147.171798,146.269416,21.1,120.122917
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,203.845886,2629400.0,4.0,-3.097331,212.767983,199.379593,40.408568,-96.756949,36.189244,208.480831,217.103342,21.1,120.122917
0,2021-10-01,AXP,168.500000,175.119995,168.479996,170.695389,3956000.0,4.0,2.281748,174.864263,149.785746,56.265103,117.565552,15.667511,161.812905,164.064442,21.1,120.122917
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,0.730320,226.909442,205.727561,51.614047,116.649440,2.027170,217.175334,221.968500,21.1,120.122917
0,2021-10-01,CAT,192.899994,195.869995,191.240005,187.928040,3695500.0,4.0,-3.640323,205.735917,181.432778,41.999441,-112.087750,36.203176,196.993865,200.522105,21.1,120.122917


In [19]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [20]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [21]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [22]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [23]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

TIMESTEPS = [50000,50000,50000,50000,50000]

### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [24]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [25]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=TIMESTEPS[0]) if if_using_a2c else None

-------------------------------------
| time/                 |           |
|    fps                | 42        |
|    iterations         | 100       |
|    time_elapsed       | 11        |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -41.2     |
|    explained_variance | 5.96e-08  |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -59.6     |
|    reward             | 1.0448978 |
|    std                | 1         |
|    value_loss         | 3.62      |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 51        |
|    iterations         | 200       |
|    time_elapsed       | 19        |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41.2     |
|    explained_variance | 0.105     |
|    learning_rate      | 0.0007    |
|    n_updat

### Agent 2: DDPG

In [26]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to results/ddpg


In [27]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=TIMESTEPS[1]) if if_using_ddpg else None

day: 2956, episode: 20
begin_total_asset: 1000000.00
end_total_asset: 5442983.33
total_reward: 4442983.33
total_cost: 1749.20
total_trades: 36503
Sharpe: 0.890
-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 24         |
|    time_elapsed    | 491        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | 8.32       |
|    critic_loss     | 55.7       |
|    learning_rate   | 0.001      |
|    n_updates       | 8871       |
|    reward          | -7.7807484 |
-----------------------------------
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 21         |
|    time_elapsed    | 1104       |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 0.904      |
|    critic_loss     | 4.34       |
|    learning_rate   | 0.001      |
|    n_updat

### Agent 3: PPO

In [28]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to results/ppo


In [29]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=TIMESTEPS[2]) if if_using_ppo else None

----------------------------------
| time/              |           |
|    fps             | 62        |
|    iterations      | 1         |
|    time_elapsed    | 32        |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1.9690489 |
----------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 61          |
|    iterations           | 2           |
|    time_elapsed         | 66          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014264977 |
|    clip_fraction        | 0.202       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.2       |
|    explained_variance   | -0.0122     |
|    learning_rate        | 0.00025     |
|    loss                 | 6.85        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0308     |
|    reward  

### Agent 4: TD3

In [30]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device
Logging to results/td3


In [31]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=TIMESTEPS[3]) if if_using_td3 else None

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 24         |
|    time_elapsed    | 485        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | -18.2      |
|    critic_loss     | 3.05e+03   |
|    learning_rate   | 0.001      |
|    n_updates       | 8871       |
|    reward          | -5.4769893 |
-----------------------------------
day: 2956, episode: 60
begin_total_asset: 1000000.00
end_total_asset: 3288091.49
total_reward: 2288091.49
total_cost: 1185.23
total_trades: 59158
Sharpe: 0.619
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 21         |
|    time_elapsed    | 1092       |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 9.12       |
|    critic_loss     | 83.9       |
|    learning_rate   | 0.001      |
|    n_updat

### Agent 5: SAC

In [32]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to results/sac


In [33]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=TIMESTEPS[4]) if if_using_sac else None

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 19         |
|    time_elapsed    | 599        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | 267        |
|    critic_loss     | 20.6       |
|    ent_coef        | 0.0815     |
|    ent_coef_loss   | -114       |
|    learning_rate   | 0.0001     |
|    n_updates       | 11727      |
|    reward          | -3.5861566 |
-----------------------------------
day: 2956, episode: 80
begin_total_asset: 1000000.00
end_total_asset: 3861827.22
total_reward: 2861827.22
total_cost: 18573.40
total_trades: 51384
Sharpe: 0.753
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 19        |
|    time_elapsed    | 1215      |
|    total_timesteps | 23656     |
| train/             |           |
|    actor_loss      | 105       |
|    critic_loss   

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [34]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [35]:
insample_risk_indicator.vix.describe()

count    2957.000000
mean       18.105293
std         7.272476
min         9.140000
25%        13.370000
50%        16.209999
75%        20.629999
max        82.690002
Name: vix, dtype: float64

In [36]:
insample_risk_indicator.vix.quantile(0.9)

57.212001831054636

In [37]:
insample_risk_indicator.turbulence.describe()

count    2957.000000
mean       34.139586
std        43.879103
min         0.000000
25%        14.613449
50%        23.644963
75%        38.292666
max       652.505968
Name: turbulence, dtype: float64

In [38]:
insample_risk_indicator.turbulence.quantile(0.9)

291.7261025697854

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [39]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = insample_risk_indicator.turbulence.quantile(0.9),risk_indicator_col='turbulence', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [40]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,141.404266,94639600.0,4.0,-1.703487,155.382845,137.132192,46.927735,-142.190143,41.749873,147.171798,146.269416,21.1,120.122917
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,203.845886,2629400.0,4.0,-3.097331,212.767983,199.379593,40.408568,-96.756949,36.189244,208.480831,217.103342,21.1,120.122917
0,2021-10-01,AXP,168.500000,175.119995,168.479996,170.695389,3956000.0,4.0,2.281748,174.864263,149.785746,56.265103,117.565552,15.667511,161.812905,164.064442,21.1,120.122917
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,0.730320,226.909442,205.727561,51.614047,116.649440,2.027170,217.175334,221.968500,21.1,120.122917
0,2021-10-01,CAT,192.899994,195.869995,191.240005,187.928040,3695500.0,4.0,-3.640323,205.735917,181.432778,41.999441,-112.087750,36.203176,196.993865,200.522105,21.1,120.122917


In [41]:
trained_moedl = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [42]:
trained_moedl = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [43]:
trained_moedl = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [44]:
trained_moedl = trained_td3
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [45]:
trained_moedl = trained_sac
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

hit end!


In [46]:
df_account_value_a2c.shape

(353, 2)

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [83]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0])
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0])
df_account_value_a2c.to_csv("results/df_account_value_a2c.csv")
df_account_value_ddpg.to_csv("results/df_account_value_ddpg.csv")
df_account_value_td3.to_csv("results/df_account_value_td3.csv")
df_account_value_ppo.to_csv("results/df_account_value_ppo.csv")
df_account_value_sac.to_csv("results/df_account_value_sac.csv")
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI", 
        start = TRADE_START_DATE,
        end = TRADE_END_DATE)
stats = backtest_stats(df_dji_, value_col_name = 'close')
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value_a2c['date']
df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
df_dji.to_csv("results/df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
df_dji.to_csv("results/df_dji+.csv")

result = pd.merge(df_result_a2c, df_result_ddpg, left_index=True, right_index=True)
result = pd.merge(result, df_result_td3, left_index=True, right_index=True)
result = pd.merge(result, df_result_ppo, left_index=True, right_index=True)
result = pd.merge(result, df_result_sac, left_index=True, right_index=True)
# result = pd.merge(result, MVO_result, left_index=True, right_index=True)
result = pd.merge(result, df_dji, left_index=True, right_index=True)
result.columns = ['a2c', 'ddpg', 'td3', 'ppo', 'sac', 'dji']

print("result: ", result)
result.to_csv("results/result.csv")

==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (354, 8)
Annual return         -0.034876
Cumulative returns    -0.048644
Annual volatility      0.181612
Sharpe ratio          -0.105351
Calmar ratio          -0.158953
Stability              0.280983
Max drawdown          -0.219408
Omega ratio            0.982546
Sortino ratio         -0.146974
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.970602
Daily value at risk   -0.022957
dtype: float64
result:                       a2c          ddpg           td3           ppo  \
2021-10-26  1.051812e+06  1.018527e+06  1.056433e+06  1.011904e+06   
2021-10-27  1.043241e+06  1.014168e+06  1.047235e+06  1.009184e+06   
2021-10-28  1.052310e+06  1.022907e+06  1.054719e+06  1.015529e+06   
2021-10-29  1.055662e+06  1.027023e+06  1.057137e+06  1.014713e+06   
2021-11-01  1.057257e+06  1.032846e+06  1.060964e+06  1.018755e+06 

<ipython-input-83-8649fecf2458>:23: FutureWarning: Passing 'suffixes' which cause duplicate columns {'account_value_x'} in the result is deprecated and will raise a MergeError in a future version.
  result = pd.merge(result, df_result_ppo, left_index=True, right_index=True)
<ipython-input-83-8649fecf2458>:26: FutureWarning: Passing 'suffixes' which cause duplicate columns {'account_value_x'} in the result is deprecated and will raise a MergeError in a future version.
  result = pd.merge(result, df_dji, left_index=True, right_index=True)


In [1]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
fig = plt.figure()
result.plot()
fig.savefig('backtest.png')


KeyboardInterrupt



In [2]:
import shutil
folder_name = 'results'
zip_file_name = 'results'
shutil.make_archive(zip_file_name, 'zip', folder_name)

'c:\\Users\\ThinkPad\\Desktop\\code\\FinRL\\results.zip'